# HTML語法分析
### Table of contents
- [第 1 章：認識 HTML Parser](#1.-html-parser)
- [練習1：容錯實驗](#Example1)
- [第 2 章：認識 requests](#2.-requests) 
- [練習2：取得網頁原始碼](#Example2)
- [第 3 章：爬行表單與登入頁](#3.-forms-and-login-pages)
- [練習3：登入實驗](#Example3)
- [第 4 章：處理 Cookie / Session](#4.-cookie-session)
- [練習4：登入網頁後](#Example4)

## 1. Html-Parser

**解析器是把 HTML 字串轉成可讀結構的工具**，常見有：

| 解析器     | 特性           |
|------------|----------------|
| html.parser | 內建、輕量    |
| lxml       | 快速、高容錯  |
| html5lib   | 最接近瀏覽器解析 |


### 1. `html.parser`（Python 內建）

- 由 Python 官方提供，無需安裝。
- 較快但容錯力普通，適合簡單 HTML。
- 背後基於 `HTMLParser` 類別。

### 2. `lxml`（快速、高容錯）

- 基於 C 的 libxml2/libxslt，非常快速。
- 容錯力強，適合大型或中等複雜度的 HTML。
- 須安裝：`pip install lxml`

### 3. `html5lib`（瀏覽器級別容錯）

- 完整實作 HTML5 標準的行為。
- 容錯最強，最慢但最穩定。
- 須安裝：`pip install html5lib`


### 一張圖先總結差異

| 解析器       | 速度     | 容錯力（錯誤 HTML） | 安裝需求     | 使用方式              |
|--------------|----------|----------------------|--------------|------------------------|
| `html.parser`| ⭐⭐⭐      | ⭐⭐                   | ✅ Python 內建 | `"html.parser"`       |
| `lxml`       | ⭐⭐⭐⭐     | ⭐⭐⭐                  | ❗需安裝 `lxml` | `"lxml"`              |
| `html5lib`   | ⭐        | ⭐⭐⭐⭐⭐               | ❗需安裝 `html5lib` | `"html5lib"`       |



<br>


### 使用建議

| 需求                             | 建議解析器     |
|----------------------------------|----------------|
| 要快、不想裝套件（簡單 HTML）   | `html.parser`  |
| 要準又快，願意安裝套件           | `lxml`         |
| HTML 結構很糟（亂標籤、沒閉合）  | `html5lib`     |


## Example1
[回到目錄](#table-of-contents)

In [2]:
!pip install lxml
!pip install html5lib

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.0 MB 5.6 MB/s eta 0:00:01
   ---------------------------- ----------- 2.9/4.0 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 8.3 MB/s eta 0:00:00


In [3]:
from bs4 import BeautifulSoup

html = """
<p>這是一段文字
<div class=section>
    <b>粗體文字
    <i>斜體開始
</div>
"""

# 使用 html.parser（Python 內建）
soup1 = BeautifulSoup(html, "html.parser")
print("html.parser：")
print(soup1.prettify())
print("=" * 50)

# 使用 lxml（速度快，自動補標籤）
soup2 = BeautifulSoup(html, "lxml")
print("lxml：")
print(soup2.prettify())
print("=" * 50)

# 使用 html5lib（完整瀏覽器行為）
soup3 = BeautifulSoup(html, "html5lib")
print("html5lib：")
print(soup3.prettify())


html.parser：
<p>
 這是一段文字
 <div class="section">
  <b>
   粗體文字
   <i>
    斜體開始
   </i>
  </b>
 </div>
</p>

lxml：
<html>
 <body>
  <p>
   這是一段文字
  </p>
  <div class="section">
   <b>
    粗體文字
    <i>
     斜體開始
    </i>
   </b>
  </div>
 </body>
</html>

html5lib：
<html>
 <head>
 </head>
 <body>
  <p>
   這是一段文字
  </p>
  <div class="section">
   <b>
    粗體文字
    <i>
     斜體開始
    </i>
   </b>
  </div>
  <b>
   <i>
   </i>
  </b>
 </body>
</html>



## 2. requests
[回到目錄](#table-of-contents)

### 套件介紹

- requests 是 Python 最常用的 HTTP 客戶端工具，它的名字就來自「HTTP request（請求）」，可以讓你像瀏覽器一樣，對網站發送請求、取得資料。

### 為什麼重要？

- 網頁爬蟲最基本的第一步：先把 HTML 抓回來。
- 能取代過去複雜的 urllib、urllib2，簡單直覺、好用。
- 當你用瀏覽器打開一個網頁，其實就是在對伺服器發送一個「GET 請求」。伺服器會把 HTML 結構（像骨架）送回來，讓瀏覽器渲染成畫面。

### 應用場景：

你想要做一個「自動天氣播報小助手」，那就要能從氣象網站抓資料。

## Example2
[回到目錄](#table-of-contents)

In [20]:
import requests
url = "https://www.google.com"
res = requests.get(url)
print(res.status_code)  # 200 表成功
print(res.text[:200])   # 印出前 200 字 HTML

# 開網頁測試

200
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="zh-TW"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/


## 3. forms and login pages

### 實際情境：

- 很多網站要登入後才能看會員資料，例如：學校選課系統、銀行明細、討論區貼文...
- 要進入這些頁面，就必須「登入」，也就是模擬瀏覽器按下登入鍵那一刻。
- 瀏覽器會把帳號密碼「POST」到登入介面。
- 伺服器核對正確後，會回傳一組 Cookie 作為「登入憑證」。

### 目前狀況：

現在大多數網站的資安防護都很完善，尤其是登入行為，多半會使用：

- JavaScript 混淆或動態產生 token
- CSRF 保護機制
- reCAPTCHA 驗證
- OAuth、JWT 身份驗證
- HTTP Only / Secure cookie…
- 這會讓「模擬登入」變得越來越難做。

## Example3
[回到目錄](#table-of-contents)

#### the-internet.herokuapp.com/login
- 功能：模擬一個簡單的登入頁，帳密如下：
- 帳號：tomsmith
- 密碼：SuperSecretPassword!

In [10]:
import requests

login_url = "https://the-internet.herokuapp.com/authenticate"
payload = {"username": "tomsmith", "password": "SuperSecretPassword!"}

session = requests.Session()
res = session.post(login_url, data=payload)

print(res.url)
print("登入成功" if "secure" in res.url else "登入失敗")


https://the-internet.herokuapp.com/secure
登入成功


## 4. Cookie Session

### 概念補充：
- Cookie 是一種存在瀏覽器中的「識別資料」。網站會在你登入後給你一張「通行證」，這就是 Cookie。
- Session 則是 server 端紀錄你的登入狀態，它會搭配 Cookie 來認得你是誰。

### 用程式自動處理：
- 使用 requests.Session() 可以模擬一個「持久的瀏覽器會話」，它會幫你：
- 自動保存 Cookies
- 自動附帶身份資訊
- 維持登入狀態

## Example4
[回到目錄](#table-of-contents)


In [11]:
import requests
from bs4 import BeautifulSoup

# Step 1：建立 Session
session = requests.Session()

# Step 2：送出登入資料
payload = {"username": "tomsmith", "password": "SuperSecretPassword!"}
res = session.post("https://the-internet.herokuapp.com/authenticate", data=payload)

# Step 3：登入後再訪問會員頁
profile = session.get("https://the-internet.herokuapp.com/secure")
soup = BeautifulSoup(profile.text, "html.parser")
title = soup.find("h2").text.strip()

# Step 4：結果判斷
if "Secure Area" in title:
    print("✅ 登入成功")
else:
    print("❌ 登入失敗")


✅ 登入成功


In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(profile.text, "html.parser")
title = soup.find("h2").text.strip()
print("頁面標題：", title)

頁面標題： Login Page


| 概念                   | 說明                    |
| -------------------- | --------------------- |
| Cookie               | 網站給瀏覽器的「識別證」          |
| Session              | 伺服器端的登入紀錄，用來對應 Cookie |
| `requests.Session()` | 建立一個模擬瀏覽器的「持久會話」      |



### 小總結：
- requests 是一次性快遞，Session 是開一間常駐分店。
- requests.get() 每次都是陌生人
- session.get() 是老顧客，網站會記得你

### 視覺化與邏輯的呈現

In [ ]:
# 此方式無法成功，只是讓同學練習一下，所以沒成功是正常的唷

import requests 
import urllib.parse 

# Step 1: 設定 User-Agent (模擬真實瀏覽器) 
 
my_headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36" } 

# Step 2: 建立 Session 並設定標頭 

session = requests.Session() 
session.headers.update(my_headers) 

# Step 3: 送出登入請求 

login_url = "https://the-internet.herokuapp.com/authenticate" 
payload = {"username": "tomsmith", "password": "SuperSecretPassword!"} 
res = session.post(login_url, data=payload) 

# Step 4: 確認登入成功並取得 Cookie 

if "secure" in res.url: 
    print("✅ Python 登入成功！") 
    print("=" * 60) 
    
    # 取得 Cookie 
    raw_cookie = session.cookies.get('rack.session') 
    clean_cookie = urllib.parse.unquote(raw_cookie) 
    print("📋 請複製以下資訊到瀏覽器：") 
    print("-" * 60) 
    print("Cookie 名稱: rack.session") 
    print(f"Cookie 值:\n{clean_cookie}") 
    print("-" * 60) 
    print(f"🌐 User-Agent:\n{my_headers['User-Agent']}") 
    print("=" * 60) 
    print("\n📝 使用方法：") 
    print("1. 打開 Chrome，按 F12 開啟開發者工具") 
    print("2. 切換到 Console 分頁") 
    print("3. 輸入以下指令（將 Cookie 值貼上）：") 
    print(' document.cookie = "rack.session=你的Cookie值; path=/"') 
    print("4. 重新整理頁面或訪問 https://the-internet.herokuapp.com/secure") 
    print("5. 你應該能直接進入 Secure Area，不需要輸入帳密！") 
else: 
    print("❌ 登入失敗") 

✅ Python 登入成功！
📋 請複製以下資訊到瀏覽器：
------------------------------------------------------------
Cookie 名稱: rack.session
Cookie 值:
BAh7CkkiD3Nlc3Npb25faWQGOgZFVEkiRWJlZDQ5OTRiYzQ4OGQ0Mzg0ZGJm
NzAzNmRhNDE4N2Q5NzBjNzQzYzNhNzFhZDk4ZGJiZTczYzY0MzQ2NTBkZjEG
OwBGSSIJY3NyZgY7AEZJIiU1YjgwODIxOGFiNmRhNTcwNWNiODViNDYxZDA2
ZDg5OAY7AEZJIg10cmFja2luZwY7AEZ7B0kiFEhUVFBfVVNFUl9BR0VOVAY7
AFRJIi00YmRjMTNjMzQyMzk0ZWVjOGJhOWNiMWFlYzNkZTc3OTFlZTdmZDBk
BjsARkkiGUhUVFBfQUNDRVBUX0xBTkdVQUdFBjsAVEkiLWRhMzlhM2VlNWU2
YjRiMGQzMjU1YmZlZjk1NjAxODkwYWZkODA3MDkGOwBGSSINdXNlcm5hbWUG
OwBGSSINdG9tc21pdGgGOwBUSSIKZmxhc2gGOwBGewA=
--1f5dcd444ecb437a411c34ab77d5543907870714
------------------------------------------------------------
🌐 User-Agent:
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36

📝 使用方法：
1. 打開 Chrome，按 F12 開啟開發者工具
2. 切換到 Console 分頁
3. 輸入以下指令（將 Cookie 值貼上）：
 document.cookie = "rack.session=你的Cookie值; path=/"
4. 重新整理頁面或訪問 https://the-internet.herokuap

### webdriver-manager 是一個自動管理瀏覽器驅動程式（Driver）的 Python 套件

自動化爬蟲中，Python 程式碼不能直接控制瀏覽器，中間需要一個「翻譯官」，也就是 Driver（例如 Chrome 需要 chromedriver）。
webdriver-manager 就是負責自動幫你請這位翻譯官過來的經紀人。

|瀏覽器|對應的 Manager|
|:-:|:-:|
|Google Chrome|ChromeDriverManager().install()|
|Firefox|GeckoDriverManager().install()|
|Microsoft Edge|EdgeChromiumDriverManager().install()|
|Opera|OperaDriverManager().install()|

In [1]:
!pip install selenium webdriver-manager

In [2]:
# 先確保已安裝 selenium 和 webdriver-manager
# !pip install selenium webdriver-manager

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

print("🔧 正在啟動 Chrome 瀏覽器...")
# 1. 啟動真正的 Chrome 瀏覽器（會自動下載驅動程式）
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 2. 前往登入頁
print("🌐 前往登入頁面...")
driver.get("https://the-internet.herokuapp.com/login")
time.sleep(2)  # 暫停讓你看清楚

# 3. 自動輸入帳號
print("⌨️ 正在輸入帳號：tomsmith")
username_input = driver.find_element(By.ID, "username")
username_input.send_keys("tomsmith")
time.sleep(1)

# 4. 自動輸入密碼
print("⌨️ 正在輸入密碼：SuperSecretPassword!")
password_input = driver.find_element(By.ID, "password")
password_input.send_keys("SuperSecretPassword!")
time.sleep(1)

# 5. 自動按下登入按鈕
print("👆 點擊登入按鈕...")
login_btn = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
login_btn.click()

# 6. 驗證結果
time.sleep(2)
if "secure" in driver.current_url:
    print("✅ 登入成功！")
    print(f"📍 當前網址：{driver.current_url}")
    print("🎉 你現在應該看到綠色的 Secure Area 頁面了！")
else:
    print("❌ 登入失敗")

# 7. 讓瀏覽器保持開啟，直到你按 Enter
print("\n⏸️ 瀏覽器將保持開啟，請觀察登入後的頁面...")
print("💡 你可以在瀏覽器中自由操作，觀察登入後的狀態")
input("按 Enter 鍵關閉瀏覽器...")
driver.quit()
print("👋 瀏覽器已關閉")

🔧 正在啟動 Chrome 瀏覽器...
🌐 前往登入頁面...
⌨️ 正在輸入帳號：tomsmith
⌨️ 正在輸入密碼：SuperSecretPassword!
👆 點擊登入按鈕...
✅ 登入成功！
📍 當前網址：https://the-internet.herokuapp.com/secure
🎉 你現在應該看到綠色的 Secure Area 頁面了！

⏸️ 瀏覽器將保持開啟，請觀察登入後的頁面...
💡 你可以在瀏覽器中自由操作，觀察登入後的狀態
👋 瀏覽器已關閉


## 學生遇到問題 Q & A

### Q1：執行 Selenium 出現錯誤是怎麼回事？

```
ImportError: cannot import name 'Sentinel' from 'typing_extensions'
```

### A1: 問題出在套件版本問題
- Selenium 需要較新的 typing_extensions
- 目前安裝的版本過舊，缺少 Sentinel

### S1: 解決方法

執行下面的指令，更新套件至最新版本
```
!pip install -U typing_extensions
```
> 注意若安裝完後還是無法正常執行，請先關掉整個IDE後再重開即可


#### 更新套件
- pip install：更新「磁碟上的套件」
- 重開 IDE / Kernel：重新載入「記憶體中的套件」
- 兩者缺一不可

> 為何有時候不需要重啟IDE也可以正常執行

> 有可能載入此套件尚未被執行到相依的地方故不影響，sys.modules查看

In [3]:
import sys
sys.modules

{'sys': <module 'sys' (built-in)>,
 'builtins': <module 'builtins' (built-in)>,
 '_frozen_importlib': <module '_frozen_importlib' (frozen)>,
 '_imp': <module '_imp' (built-in)>,
 '_thread': <module '_thread' (built-in)>,
 '_warnings': <module '_warnings' (built-in)>,
 '_weakref': <module '_weakref' (built-in)>,
 'winreg': <module 'winreg' (built-in)>,
 '_io': <module '_io' (built-in)>,
 'marshal': <module 'marshal' (built-in)>,
 'nt': <module 'nt' (built-in)>,
 '_frozen_importlib_external': <module '_frozen_importlib_external' (frozen)>,
 'time': <module 'time' (built-in)>,
 'zipimport': <module 'zipimport' (frozen)>,
 '_codecs': <module '_codecs' (built-in)>,
 'codecs': <module 'codecs' (frozen)>,
 'encodings.aliases': <module 'encodings.aliases' from 'c:\\Users\\USER\\anaconda3\\Lib\\encodings\\aliases.py'>,
 'encodings': <module 'encodings' from 'c:\\Users\\USER\\anaconda3\\Lib\\encodings\\__init__.py'>,
 'encodings.utf_8': <module 'encodings.utf_8' from 'c:\\Users\\USER\\anaconda3\